Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import datetime

In [2]:
pickle_file = 'notMNIST_cut.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (195107, 28, 28) (195107,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (195107, 28, 28, 1) (195107, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 32
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    -

In [6]:
num_steps = 1001
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                  valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 2.988412
Minibatch accuracy: 12.5%
Validation accuracy: 10.4%
Minibatch loss at step 100: 1.041782
Minibatch accuracy: 62.5%
Validation accuracy: 71.5%
Minibatch loss at step 200: 0.885709
Minibatch accuracy: 81.2%
Validation accuracy: 77.9%
Minibatch loss at step 300: 0.707019
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.771516
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.373252
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%
Minibatch loss at step 600: 0.594556
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 700: 0.552307
Minibatch accuracy: 75.0%
Validation accuracy: 82.1%
Minibatch loss at step 800: 0.455444
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Minibatch loss at step 900: 1.130393
Minibatch accuracy: 68.8%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 0.841953
Minibatch accuracy: 68.8%
Validation accuracy: 83.5%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
depth = 1
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 2 * image_size // 2 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        pool = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
        hidden = tf.nn.relu(pool)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 100001
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                  valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 2.713693
Minibatch accuracy: 6.2%
Validation accuracy: 9.4%
Minibatch loss at step 10000: 0.613330
Minibatch accuracy: 87.5%
Validation accuracy: 83.6%
Minibatch loss at step 20000: 0.861383
Minibatch accuracy: 87.5%
Validation accuracy: 84.5%
Minibatch loss at step 30000: 1.032659
Minibatch accuracy: 62.5%
Validation accuracy: 85.1%
Minibatch loss at step 40000: 0.936007
Minibatch accuracy: 68.8%
Validation accuracy: 85.5%
Minibatch loss at step 50000: 0.373681
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 60000: 0.500959
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 70000: 0.390476
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 80000: 0.161248
Minibatch accuracy: 93.8%
Validation accuracy: 86.7%
Minibatch loss at step 90000: 0.663680
Minibatch accuracy: 75.0%
Validation accuracy: 86.7%
Minibatch loss at step 100000: 0.146792
Minibatch accuracy: 93.8%
Validati

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [37]:
batch_size = 16
depth = 1
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.1
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               100000, 0.01, staircase=True)
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 2 * image_size // 2 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], 'VALID')
        hidden = tf.nn.relu(pool)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits)) 
            + 0.00001 * tf.nn.l2_loss(layer3_weights) 
            + 0.00001 * tf.nn.l2_loss(layer4_weights))
    
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(
        learning_rate).minimize(loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [38]:
num_steps = 140001
t0 = datetime.datetime.now()

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for global_step in range(num_steps):
        offset = (global_step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (global_step % 20000 == 0):
            print('Minibatch loss at step %d: %f' % (global_step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                  valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

print ('Time: ', datetime.datetime.now() - t0)

Initialized
Minibatch loss at step 0: 2.430272
Minibatch accuracy: 12.5%
Validation accuracy: 10.3%
Minibatch loss at step 20000: 0.803167
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Minibatch loss at step 40000: 0.772367
Minibatch accuracy: 68.8%
Validation accuracy: 86.8%
Minibatch loss at step 60000: 0.440196
Minibatch accuracy: 81.2%
Validation accuracy: 87.0%
Minibatch loss at step 80000: 0.091040
Minibatch accuracy: 100.0%
Validation accuracy: 86.6%
Minibatch loss at step 100000: 0.186315
Minibatch accuracy: 93.8%
Validation accuracy: 86.9%
Minibatch loss at step 120000: 0.745123
Minibatch accuracy: 81.2%
Validation accuracy: 87.4%
Minibatch loss at step 140000: 0.036997
Minibatch accuracy: 100.0%
Validation accuracy: 87.5%
Test accuracy: 93.0%
Time:  0:02:40.749716
